# Analisando dados dos resultados da eleição de 2016 e 2018

OBS:
O arquivo 'votacao_candidato_munzona_2018_RJ.csv' pode ser econtrado nesses 2 links:
- [Google Drive](https://drive.google.com/open?id=1gWPqhJKlmBJm3z0d118BzCsgyCtnJRbV)
- [Eleições 2018 - Votação nominal por município e zona (formato ZIP)](http://agencia.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2018.zip)

O arquivo 'votacao_candidato_munzona_2016_RJ.csv' pode ser econtrado nesses 2 links:
- [Google Drive](https://drive.google.com/open?id=181vRDyOuxAjxF3TTI3fK9dfcjLR8f3IA)
- [Eleições 2016 - Votação nominal por município e zona (formato ZIP)](http://agencia.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2016.zip)

Não foi possível subir aqui no Github devido aos tamanhos do arquivos.

In [1]:
import pandas as pd

In [2]:
zonas = pd.read_csv('dados/bairros_zonas.csv')
zonas.head()

,Zona,Bairro
0,4,BOTAFOGO
1,4,HUMAITA
2,4,FLAMENGO
3,4,URCA
4,5,COPACABA\rNA


In [3]:
resultados_2016 = pd.read_csv('dados/votacao_candidato_munzona_2016_RJ.csv', delimiter=';', encoding = "ISO-8859-1")
resultados_2016.head(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_VOTO_EM_TRANSITO,QT_VOTOS_NOMINAIS
0,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,36,PTC,Partido Trabalhista Cristão,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0
1,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,70,PT do B,Partido Trabalhista do Brasil,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0
2,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,70,PT do B,Partido Trabalhista do Brasil,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0


In [4]:
resultados_2018 = pd.read_csv('dados/votacao_candidato_munzona_2018_RJ.csv', delimiter=';', encoding = "ISO-8859-1")
resultados_2018.head(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_VOTO_EM_TRANSITO,QT_VOTOS_NOMINAIS
0,16/11/2019,18:45:32,2018,2,Eleição Ordinária,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,45,PSDB,Partido da Social Democracia Brasileira,190000050094,O RIO TEM JEITO,SOLIDARIEDADE / PSDB / PPS,5,SUPLENTE,N,22
1,16/11/2019,18:45:32,2018,2,Eleição Ordinária,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,31,PHS,Partido Humanista da Solidariedade,190000050173,PARTIDO ISOLADO,PHS,3,ELEITO POR MÉDIA,N,2
2,16/11/2019,18:45:32,2018,2,Eleição Ordinária,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,12,PDT,Partido Democrático Trabalhista,190000050590,PARTIDO ISOLADO,PDT,5,SUPLENTE,N,12


**Definindo funções**

In [5]:
def filtrar_eleitos_municipio(df, municipio):
    
    return df.loc[(df['NM_MUNICIPIO'] == municipio) & 
               ((df['DS_SIT_TOT_TURNO'] == 'ELEITO') |
               (df['DS_SIT_TOT_TURNO'] == 'ELEITO POR MÉDIA') |
               (df['DS_SIT_TOT_TURNO'] == 'ELEITO POR QP'))][['ANO_ELEICAO', 'NR_TURNO', 
              'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO', 
              'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'DS_SITUACAO_CANDIDATURA', 'SG_PARTIDO', 
              'NM_PARTIDO', 'QT_VOTOS_NOMINAIS', 'ST_VOTO_EM_TRANSITO', 
              'DS_SIT_TOT_TURNO']]

In [6]:
def get_total_votos(df):
    
    total_votos = df.groupby('NM_CANDIDATO').agg({'QT_VOTOS_NOMINAIS': 'sum'}).reset_index()
    total_votos = total_votos.rename(columns={"QT_VOTOS_NOMINAIS": "TOTAL_VOTOS"})
    
    return total_votos

In [7]:
def add_total_votos(df):
    
    total_votos = get_total_votos(df)
    df = df.merge(total_votos, on='NM_CANDIDATO')
    
    return df

In [8]:
def add_porcentagem_votos(df):
    
    df['PROPORCAO'] = df['QT_VOTOS_NOMINAIS']/df['TOTAL_VOTOS']
    df['PORCENTAGEM'] = df['PROPORCAO']*100
    
    return df

In [9]:
def filtrar_top_votos(df, indice):
    return df.sort_values(['NM_CANDIDATO', 'QT_VOTOS_NOMINAIS'], 
                          ascending=False).groupby('NM_CANDIDATO').head(indice)

In [33]:
def filtrar_top_votados_zona(df, indice):
    return df.sort_values(['NR_ZONA', 'CD_CARGO', 'QT_VOTOS_NOMINAIS'], 
                          ascending=False).groupby(['NR_ZONA', 'CD_CARGO']).head(indice)

In [23]:
def gerar_top_votacoes_candidato(df, indice, municipio):
    
    df = filtrar_eleitos_municipio(df, municipio)
    df = add_total_votos(df)
    df = filtrar_top_votos(df, indice)
    df = add_porcentagem_votos(df)
    
    return df

In [24]:
def gerar_top_votacoes_zona(df, indice, municipio):
    
    df = filtrar_eleitos_municipio(df, municipio)
    df = add_total_votos(df)
    df = filtrar_top_votados_zona(df, indice)
    df = add_porcentagem_votos(df)
    
    return df

## Eleições 2016
### Filtrando top 5 zonas por candidats no Rio de Janeiro

Cargos votados e quantidade de candidats

In [25]:
resultados_2016.groupby('DS_CARGO').agg({'SQ_CANDIDATO': 'count'})

,SQ_CANDIDATO
DS_CARGO,
Prefeito,2302
Vereador,212500


Gerando o top zonas por candidats

In [44]:
top_zonas_candidats_2016 = gerar_top_votacoes_candidato(resultados_2016, 5, 'RIO DE JANEIRO')
top_zonas_candidats_2016.sort_values(['NM_CANDIDATO', 'PORCENTAGEM'], ascending=False).head(10)

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
2427,2016,1,RIO DE JANEIRO,25,13,Vereador,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,11081,N,ELEITO POR QP,20678,0.535884,53.588355
2426,2016,1,RIO DE JANEIRO,241,13,Vereador,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,1697,N,ELEITO POR QP,20678,0.082068,8.206790
2510,2016,1,RIO DE JANEIRO,243,13,Vereador,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,1184,N,ELEITO POR QP,20678,0.057259,5.725892
2492,2016,1,RIO DE JANEIRO,240,13,Vereador,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,791,N,ELEITO POR QP,20678,0.038253,3.825322
2505,2016,1,RIO DE JANEIRO,125,13,Vereador,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,671,N,ELEITO POR QP,20678,0.032450,3.244995
285,2016,1,RIO DE JANEIRO,246,13,Vereador,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,556,N,ELEITO POR MÉDIA,19946,0.027875,2.787526
209,2016,1,RIO DE JANEIRO,182,13,Vereador,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,549,N,ELEITO POR MÉDIA,19946,0.027524,2.752432
281,2016,1,RIO DE JANEIRO,9,13,Vereador,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,520,N,ELEITO POR MÉDIA,19946,0.026070,2.607039
210,2016,1,RIO DE JANEIRO,185,13,Vereador,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,489,N,ELEITO POR MÉDIA,19946,0.024516,2.451619
262,2016,1,RIO DE JANEIRO,180,13,Vereador,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,475,N,ELEITO POR MÉDIA,19946,0.023814,2.381430


**Testando resultados**

In [47]:
top_zonas_candidats_2016.loc[top_votos_candidats_2016['NM_URNA_CANDIDATO'] == 'MARIELLE FRANCO'][['NR_ZONA',
                                                                        'NM_CANDIDATO', 
                                                                        'QT_VOTOS_NOMINAIS',
                                                                        'TOTAL_VOTOS', 
                                                                        'PROPORCAO',
                                                                        'PORCENTAGEM'
                                                                       ]]

,NR_ZONA,NM_CANDIDATO,QT_VOTOS_NOMINAIS,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
4464,16,MARIELLE FRANCISCO DA SILVA,2237,46502,0.048105,4.810546
4487,4,MARIELLE FRANCISCO DA SILVA,1926,46502,0.041418,4.141757
4501,7,MARIELLE FRANCISCO DA SILVA,1837,46502,0.039504,3.950368
4481,161,MARIELLE FRANCISCO DA SILVA,1688,46502,0.036300,3.629951
4514,9,MARIELLE FRANCISCO DA SILVA,1333,46502,0.028665,2.866543


In [48]:
zonas.loc[(zonas['Zona'] == 16) | 
          (zonas['Zona'] == 4) | 
          (zonas['Zona'] == 7) | 
          (zonas['Zona'] == 161) | 
          (zonas['Zona'] == 9)]

,Zona,Bairro
0,4,BOTAFOGO
1,4,HUMAITA
2,4,FLAMENGO
3,4,URCA
6,7,TIJUCA
15,9,BARRA DA\rTIJUCA
16,9,CAMORIM
17,9,VARGEM\rGRANDE
18,9,VARGEM\rPEQUENA
19,9,RECREIO


Resultados estão de acordo com a notícias publicada pela Folha de [São Paulo](https://www1.folha.uol.com.br/cotidiano/2018/03/marielle-franco-foi-mais-votada-em-bairros-nobres-do-rio-de-janeiro.shtml)

Salvando dados no formato .csv

In [49]:
top_zonas_candidats_2016.to_csv('dados/top_5_zonas_candidats_2016_rj.csv', index=False)

## Eleições 2016

### Filtrando top 5 candidats por zona no Rio de Janeiro

In [51]:
top_candidats_zona_2016 = gerar_top_votacoes_zona(resultados_2016, 5, 'RIO DE JANEIRO')
top_candidats_zona_2016.sort_values(['NR_ZONA', 'CD_CARGO', 'PORCENTAGEM'], ascending=False).head(15)

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
3199,2016,1,RIO DE JANEIRO,252,13,Vereador,190000007443,LEANDRO LYRA BRAGA DOGNINI,LEANDRO LYRA,APTO,NOVO,Partido Novo,1405,N,ELEITO POR QP,29217,0.048088,4.808844
4471,2016,1,RIO DE JANEIRO,252,13,Vereador,190000003414,MARIELLE FRANCISCO DA SILVA,MARIELLE FRANCO,APTO,PSOL,Partido Socialismo e Liberdade,1011,N,ELEITO POR QP,46502,0.021741,2.174100
4262,2016,1,RIO DE JANEIRO,252,13,Vereador,190000003426,TARCÍSIO MOTTA DE CARVALHO,TARCÍSIO MOTTA,APTO,PSOL,Partido Socialismo e Liberdade,1293,N,ELEITO POR QP,90473,0.014292,1.429156
2909,2016,1,RIO DE JANEIRO,252,13,Vereador,190000005654,CESAR EPITÁCIO MAIA,CESAR MAIA,APTO,DEM,Democratas,867,N,ELEITO POR QP,71468,0.012131,1.213130
3435,2016,1,RIO DE JANEIRO,252,13,Vereador,190000011815,CARLOS NANTES BOLSONARO,CARLOS BOLSONARO,APTO,PSC,Partido Social Cristão,758,N,ELEITO POR QP,106657,0.007107,0.710689
4932,2016,2,RIO DE JANEIRO,252,11,Prefeito,190000017952,MARCELO BEZERRA CRIVELLA,CRIVELLA,APTO,PRB,Partido Republicano Brasileiro,10960,N,ELEITO,1700030,0.006447,0.644695
4977,2016,1,RIO DE JANEIRO,246,13,Vereador,190000010764,GILBERTO DE OLIVEIRA LIMA,DR GILBERTO,APTO,PMN,Partido da Mobilização Nacional,1673,N,ELEITO POR QP,12165,0.137526,13.752569
1858,2016,1,RIO DE JANEIRO,246,13,Vereador,190000012852,TADEU AMORIM DE BARROS JUNIOR,JUNIOR DA LUCINHA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,6160,N,ELEITO POR QP,45124,0.136513,13.651272
1508,2016,1,RIO DE JANEIRO,246,13,Vereador,190000017884,JOÃO MENDES DE JESUS,JOÃO MENDES DE JESUS,APTO,PRB,Partido Republicano Brasileiro,1737,N,ELEITO POR QP,31516,0.055115,5.511486
3477,2016,1,RIO DE JANEIRO,246,13,Vereador,190000011815,CARLOS NANTES BOLSONARO,CARLOS BOLSONARO,APTO,PSC,Partido Social Cristão,2167,N,ELEITO POR QP,106657,0.020317,2.031747


In [60]:
top_candidats_zona_2016.loc[top_candidats_zona_2016['NR_ZONA'] == 7].sort_values(['PORCENTAGEM'], ascending=False)

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
3178,2016,1,RIO DE JANEIRO,7,13,Vereador,190000007443,LEANDRO LYRA BRAGA DOGNINI,LEANDRO LYRA,APTO,NOVO,Partido Novo,1201,N,ELEITO POR QP,29217,0.041106,4.110621
4501,2016,1,RIO DE JANEIRO,7,13,Vereador,190000003414,MARIELLE FRANCISCO DA SILVA,MARIELLE FRANCO,APTO,PSOL,Partido Socialismo e Liberdade,1837,N,ELEITO POR QP,46502,0.039504,3.950368
4247,2016,1,RIO DE JANEIRO,7,13,Vereador,190000003426,TARCÍSIO MOTTA DE CARVALHO,TARCÍSIO MOTTA,APTO,PSOL,Partido Socialismo e Liberdade,3424,N,ELEITO POR QP,90473,0.037846,3.784555
2906,2016,1,RIO DE JANEIRO,7,13,Vereador,190000005654,CESAR EPITÁCIO MAIA,CESAR MAIA,APTO,DEM,Democratas,2330,N,ELEITO POR QP,71468,0.032602,3.260200
3419,2016,1,RIO DE JANEIRO,7,13,Vereador,190000011815,CARLOS NANTES BOLSONARO,CARLOS BOLSONARO,APTO,PSC,Partido Social Cristão,2025,N,ELEITO POR QP,106657,0.018986,1.898610
4859,2016,2,RIO DE JANEIRO,7,11,Prefeito,190000017952,MARCELO BEZERRA CRIVELLA,CRIVELLA,APTO,PRB,Partido Republicano Brasileiro,16115,N,ELEITO,1700030,0.009479,0.947924


In [59]:
zonas.loc[zonas['Zona'] == 7]

,Zona,Bairro
6,7,TIJUCA


Salvando dados no formato .csv

In [61]:
top_candidats_zona_2016.to_csv('dados/top_5_candidats_zona_2016.csv', index=False)

## Eleições 2018

### Filtrando top 5 zonas por candidats no Rio de Janeiro

In [62]:
resultados_2018.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO', 'ST_VOTO_EM_TRANSITO',
       'QT_VOTOS_NOMINAIS'],
      dtype='object')

In [63]:
resultados_2018.groupby('DS_CARGO').agg({'SQ_CANDIDATO': 'count'})

,SQ_CANDIDATO
DS_CARGO,
Deputado Estadual,432612
Deputado Federal,202947
Governador,2196
Senador,2745


In [73]:
top_zona_cadidatos_2018 = gerar_top_votacoes_candidato(resultados_2018, 5, 'RIO DE JANEIRO')
top_zona_cadidatos_2018.sort_values(['NM_CANDIDATO', 'PORCENTAGEM'], ascending=False).head(10)

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
640,2018,1,RIO DE JANEIRO,22,6,Deputado Federal,190000624672,WLADIMIR BARROS ASSED MATHEUS DE OLIVEIRA,WLADIMIR GAROTINHO,APTO,PRP,Partido Republicano Progressista,21,N,ELEITO POR QP,279,0.075269,7.526882
637,2018,1,RIO DE JANEIRO,16,6,Deputado Federal,190000624672,WLADIMIR BARROS ASSED MATHEUS DE OLIVEIRA,WLADIMIR GAROTINHO,APTO,PRP,Partido Republicano Progressista,16,N,ELEITO POR QP,279,0.057348,5.734767
669,2018,1,RIO DE JANEIRO,4,6,Deputado Federal,190000624672,WLADIMIR BARROS ASSED MATHEUS DE OLIVEIRA,WLADIMIR GAROTINHO,APTO,PRP,Partido Republicano Progressista,14,N,ELEITO POR QP,279,0.050179,5.017921
666,2018,1,RIO DE JANEIRO,179,6,Deputado Federal,190000624672,WLADIMIR BARROS ASSED MATHEUS DE OLIVEIRA,WLADIMIR GAROTINHO,APTO,PRP,Partido Republicano Progressista,13,N,ELEITO POR QP,279,0.046595,4.659498
646,2018,1,RIO DE JANEIRO,119,6,Deputado Federal,190000624672,WLADIMIR BARROS ASSED MATHEUS DE OLIVEIRA,WLADIMIR GAROTINHO,APTO,PRP,Partido Republicano Progressista,12,N,ELEITO POR QP,279,0.043011,4.301075
5019,2018,2,RIO DE JANEIRO,238,3,Governador,190000612301,WILSON JOSÉ WITZEL,WILSON WITZEL,APTO,PSC,Partido Social Cristão,45258,N,ELEITO,1520888,0.029758,2.975762
5015,2018,2,RIO DE JANEIRO,245,3,Governador,190000612301,WILSON JOSÉ WITZEL,WILSON WITZEL,APTO,PSC,Partido Social Cristão,43949,N,ELEITO,1520888,0.028897,2.889693
5033,2018,2,RIO DE JANEIRO,188,3,Governador,190000612301,WILSON JOSÉ WITZEL,WILSON WITZEL,APTO,PSC,Partido Social Cristão,43690,N,ELEITO,1520888,0.028727,2.872664
5001,2018,2,RIO DE JANEIRO,233,3,Governador,190000612301,WILSON JOSÉ WITZEL,WILSON WITZEL,APTO,PSC,Partido Social Cristão,42205,N,ELEITO,1520888,0.027750,2.775024
5025,2018,2,RIO DE JANEIRO,22,3,Governador,190000612301,WILSON JOSÉ WITZEL,WILSON WITZEL,APTO,PSC,Partido Social Cristão,40744,N,ELEITO,1520888,0.026790,2.678961


Salvando no csv

In [74]:
top_zona_cadidatos_2018.to_csv('dados/top_5_zona_cadidatos_2018.csv', index=False)

## Eleições 2018

### Filtrando top 5 candidats por zona no Rio de Janeiro

In [70]:
top_cadidatos_zona_2018 = gerar_top_votacoes_zona(resultados_2018, 5, 'RIO DE JANEIRO')
top_cadidatos_zona_2018.sort_values(['NR_ZONA', 'CD_CARGO', 'PORCENTAGEM'], ascending=False).head(15)

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
3250,2018,1,RIO DE JANEIRO,246,7,Deputado Estadual,190000611209,ROSANGELA DE OLIVEIRA ZEIDAN,ZEIDAN,APTO,PT,Partido dos Trabalhadores,1230,N,ELEITO POR QP,6314,0.194805,19.480519
249,2018,1,RIO DE JANEIRO,246,7,Deputado Estadual,190000608704,LUCIA HELENA PINTO DE BARROS,LUCINHA,APTO,PSDB,Partido da Social Democracia Brasileira,8716,N,ELEITO POR QP,63041,0.138259,13.825923
3151,2018,1,RIO DE JANEIRO,246,7,Deputado Estadual,190000622243,THIAGO PAMPOLHA GONCALVES,THIAGO PAMPOLHA,APTO,PDT,Partido Democrático Trabalhista,2017,N,ELEITO POR QP,29090,0.069337,6.933654
4009,2018,1,RIO DE JANEIRO,246,7,Deputado Estadual,190000626477,JUCELIA OLIVEIRA FREITAS,TIA JU,APTO,PRB,Partido Republicano Brasileiro,1400,N,ELEITO POR QP,46533,0.030086,3.008618
2930,2018,1,RIO DE JANEIRO,246,7,Deputado Estadual,190000614774,RODRIGO MARTINS PIRES DE AMORIM,RODRIGO AMORIM,APTO,PSL,Partido Social Liberal,1195,N,ELEITO POR QP,69644,0.017159,1.715869
4853,2018,1,RIO DE JANEIRO,246,6,Deputado Federal,190000607923,FLORDELIS DOS SANTOS DE SOUZA,FLORDELIS,APTO,PSD,Partido Social Democrático,2061,N,ELEITO POR QP,62403,0.033027,3.302726
1224,2018,1,RIO DE JANEIRO,246,6,Deputado Federal,190000607836,OTONI MOURA DE PAULO JUNIOR,OTONI DE PAULA,APTO,PSC,Partido Social Cristão,1424,N,ELEITO POR QP,47613,0.029908,2.990780
5258,2018,1,RIO DE JANEIRO,246,6,Deputado Federal,190000614709,HELIO FERNANDO BARBOSA LOPES,HELIO FERNANDO BARBOSA LOPES,APTO,PSL,Partido Social Liberal,2786,N,ELEITO POR QP,176273,0.015805,1.580503
1251,2018,1,RIO DE JANEIRO,246,6,Deputado Federal,190000615553,ALESSANDRO LUCCIOLA MOLON,ALESSANDRO MOLON,APTO,PSB,Partido Socialista Brasileiro,1516,N,ELEITO POR QP,144906,0.010462,1.046195
5734,2018,1,RIO DE JANEIRO,246,6,Deputado Federal,190000602109,MARCELO RIBEIRO FREIXO,MARCELO FREIXO,APTO,PSOL,Partido Socialismo e Liberdade,2227,N,ELEITO POR QP,221191,0.010068,1.006822


salvando csv

In [71]:
top_cadidatos_zona_2018.to_csv('dados/top_5_cadidatos_zona_2018.csv', index=False)